In [1]:
#匯入套件
import requests
import pandas as pd
import json
import re
from lxml import etree
import time

#函式部分
WEBSITE_URL = "https://www.sfclinic.com.tw/"
###爬取醫生info
def get_doctor_detail(doctor_url):
    resp = requests.get(doctor_url)
    root = etree.HTML(resp.content.decode('utf-8'))
    doctor_result = {
        "doc_pic":WEBSITE_URL+"".join(root.xpath("//div[@class='box']//img/@data-lazy")), 
        "doc_name":"".join(root.xpath("//h2[@class='ti']/span/text()")),
        "cli_name":''.join(root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'服務館別')]/following-sibling::div/text()")),
        "doc_prof":''.join(root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'專長')]/following-sibling::div/text()")),
        "countries":"",
        "cli_addr":"",
        "cli_phone":"",
        "cli_link":"",
        "doc_exp":''.join(root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'經歷')]/following-sibling::div/text()")),
        "doc_edu":''.join(root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'學歷')]/following-sibling::div/text()"))
    }
    return doctor_result
###爬取醫生
def get_doctors():
    resp = requests.get( f"{WEBSITE_URL}/team.php")
    root = etree.HTML(resp.content.decode('utf-8'))
    doctor_urls = root.xpath("//div[contains(@class,'list-box')]/div/a/@href")
    return list(map(lambda i: f"{WEBSITE_URL}/{i}", doctor_urls))
###爬取診所info
def get_clinic_detail(clinic_url):
    resp = requests.get(clinic_url)
    root = etree.HTML(resp.content.decode('utf-8'))
    clinic_result = {
        "countries":''.join(root.xpath("//h2[@class='store-name']/span/text()")),
        "cli_addr":''.join(root.xpath("//div[@class='address list']//i/following-sibling::text()")),
        "cli_phone":''.join(root.xpath("//div[@class='tel list']//i/following-sibling::text()")),
        "cli_link":''.join(clinic_url)
                            }
    return clinic_result
###爬取診所
def get_clinics():
    resp = requests.get( f"{WEBSITE_URL}/location.php")
    root = etree.HTML(resp.content.decode('utf-8'))
    clinic_urls =root.xpath("//div[contains(@class,'list-box')]/div/a/@href")
    return list(map(lambda i: f"{WEBSITE_URL}/{i}",clinic_urls))         
###診所資料﹑診所合併
clinic_list = get_clinics()
clinic_details = []
for clinic_idx,clinic_url in enumerate(clinic_list):
    print(f"{clinic_idx+1}/{len(clinic_list)}, {clinic_url}")
    clinic_details.append(get_clinic_detail(clinic_url))
##刪除不必要資訊
# for i in range(len(clinic_details)):
#     for i_countries in range(len(clinic_details[i]['countries'])):
#         clinic_details[i]['countries']=[sub.replace('館', '') for sub in clinic_details[i]['countries']]
###醫生資料﹑醫生合併        
doctor_list = get_doctors()
doctor_details = []
for doctor_idx,doctor_url in enumerate(doctor_list):
    print(f"{doctor_idx+1}/{len(doctor_list)}, {doctor_url}")
    doctor_details.append(get_doctor_detail(doctor_url))
###診所﹑醫生合併   
def mergeinfo():
    for i in range(len(doctor_details)):
        for j in range(len(clinic_details)):
            if clinic_details[j]['countries'] in doctor_details[i]['cli_name']:
                doctor_details[i]['countries']=doctor_details[i]['countries']+",".join(clinic_details[j]['countries'])
                doctor_details[i]['cli_addr']=doctor_details[i]['cli_addr']+",".join(clinic_details[j]['cli_addr'])
                doctor_details[i]['cli_phone']=doctor_details[i]['cli_phone']+",".join(clinic_details[j]['cli_phone'])
                doctor_details[i]['cli_link']=doctor_details[i]['cli_link']+",".join(clinic_details[j]['cli_link'])
    return doctor_details
    

#執行程式
FILE_NAME = "順風醫美診所"
today = time.strftime("%Y%m%d")
df = pd.DataFrame(mergeinfo(), columns=["doc_pic","doc_name","cli_name","countries","cli_addr","cli_phone","cli_link","doc_prof","doc_exp","doc_edu"])
df.to_csv(f"{FILE_NAME}-{today}.csv",index=False)

1/6, https://www.sfclinic.com.tw//location.php?act=view&id=2
2/6, https://www.sfclinic.com.tw//location.php?act=view&id=1
3/6, https://www.sfclinic.com.tw//location.php?act=view&id=5
4/6, https://www.sfclinic.com.tw//location.php?act=view&id=6
5/6, https://www.sfclinic.com.tw//location.php?act=view&id=7
6/6, https://www.sfclinic.com.tw//location.php?act=view&id=4
1/31, https://www.sfclinic.com.tw//team.php?act=view&id=65
2/31, https://www.sfclinic.com.tw//team.php?act=view&id=64
3/31, https://www.sfclinic.com.tw//team.php?act=view&id=63
4/31, https://www.sfclinic.com.tw//team.php?act=view&id=62
5/31, https://www.sfclinic.com.tw//team.php?act=view&id=61
6/31, https://www.sfclinic.com.tw//team.php?act=view&id=60
7/31, https://www.sfclinic.com.tw//team.php?act=view&id=58
8/31, https://www.sfclinic.com.tw//team.php?act=view&id=56
9/31, https://www.sfclinic.com.tw//team.php?act=view&id=54
10/31, https://www.sfclinic.com.tw//team.php?act=view&id=50
11/31, https://www.sfclinic.com.tw//team.ph